In [1]:
texto = '''A Apple foi fundada em 1 de abril de 1976 por Steve Wozniak, Steve Jobs e Ronald Wayne 
com o nome de Apple Computers, na Califórnia. O nome foi escolhido por Jobs após a visita do pomar 
de maçãs da fazenda de Robert Friedland, também pelo fato do nome soar bem e ficar antes da Atari 
nas listas telefônicas.

O primeiro protótipo da empresa foi o Apple I que foi demonstrado na Homebrew Computer Club em 1975, 
as vendas começaram em julho de 1976 com o preço de US$ 666,66, aproximadamente 200 unidades foram 
vendidas,[21] em 1977 a empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'''

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List 
from langchain_core.utils.function_calling import convert_to_openai_function

class Acontecimento(BaseModel):
    """Informação sobre um acontecimento"""
    data:str = Field(description="Data do acontecimento no formato dd/mm/aaaa")
    acontecimento:str = Field(description='Acontecimento extraído do texto')
    
class ListaAcontecimentos(BaseModel):
    """Acontecimentos para extração"""
    acontecimentos: List[Acontecimento] = Field(description="Lista de acontecimentos presentes no texto informado")
    
tool_acontecimentos = convert_to_openai_function(ListaAcontecimentos)
tool_acontecimentos

{'name': 'ListaAcontecimentos',
 'description': 'Acontecimentos para extração',
 'parameters': {'type': 'object',
  'properties': {'acontecimentos': {'description': 'Lista de acontecimentos presentes no texto informado',
    'type': 'array',
    'items': {'description': 'Informação sobre um acontecimento',
     'type': 'object',
     'properties': {'data': {'description': 'Data do acontecimento no formato dd/mm/aaaa',
       'type': 'string'},
      'acontecimento': {'description': 'Acontecimento extraído do texto',
       'type': 'string'}},
     'required': ['data', 'acontecimento']}}},
  'required': ['acontecimentos']}}

In [3]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 

prompt = ChatPromptTemplate.from_messages([
	("system", "Extraia as frases de acontecimentos. Elas devem ser extraídas integralmente"),
	("user", "{input}")
])

chat = ChatOpenAI() 

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}))

In [4]:
chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"acontecimentos":[{"data":"1 de abril de 1976","acontecimento":"A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne na Califórnia com o nome de Apple Computers."},{"data":"1975","acontecimento":"Demonstração do primeiro protótipo da empresa, o Apple I, na Homebrew Computer Club."},{"data":"julho de 1976","acontecimento":"Início das vendas do Apple I por US$ 666,66, com aproximadamente 200 unidades vendidas."},{"data":"1977","acontecimento":"A empresa recebeu o aporte de Mike Markkula e um empréstimo do Bank of America."}]}', 'name': 'ListaAcontecimentos'}}, response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 325, 'total_tokens': 478}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-28255589-004e-4bbb-aa3c-a48dfd187cc5-0', usage_metadata={'input_tokens': 325, 'output_tokens': 153, 'total_tokens': 478})

In [5]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | chat.bind(
        functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}
    )
    | JsonOutputFunctionsParser()
)

chain.invoke({"input":texto})

{'acontecimentos': [{'data': '1 de abril de 1976',
   'acontecimento': 'A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne na Califórnia com o nome de Apple Computers.'},
  {'data': '1975',
   'acontecimento': 'O primeiro protótipo da empresa, o Apple I, foi demonstrado na Homebrew Computer Club.'},
  {'data': 'julho de 1976',
   'acontecimento': 'As vendas do Apple I começaram com o preço de US$ 666,66.'},
  {'data': '1977',
   'acontecimento': 'A empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'}]}

In [7]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | chat.bind(
        functions=[tool_acontecimentos], function_call={"name": "ListaAcontecimentos"}
    )
    | JsonKeyOutputFunctionsParser(key_name="acontecimentos")
)

chain.invoke({"input":texto})

[{'data': '1 de abril de 1976',
  'acontecimento': 'A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne na Califórnia, com o nome de Apple Computers.'},
 {'data': '1975',
  'acontecimento': 'Demonstração do primeiro protótipo da empresa, o Apple I, na Homebrew Computer Club.'},
 {'data': 'julho de 1976',
  'acontecimento': 'Início das vendas do Apple I com o preço de US$ 666,66, aproximadamente 200 unidades vendidas.'},
 {'data': '1977',
  'acontecimento': 'A empresa recebeu o aporte de Mike Markkula e um empréstimo do Bank of America.'}]

In [8]:
from langchain_community.document_loaders.web_base import WebBaseLoader 

loader = WebBaseLoader('https://hub.asimov.academy/blog/')
page = loader.load()
page

[Document(metadata={'source': 'https://hub.asimov.academy/blog/', 'title': 'Blog | Asimov Academy', 'language': 'pt-BR'}, page_content='\n\n\n\n\n\n\n\n\n\n\nBlog | Asimov Academy\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEntrar\nCadastrar\n\n\n\n\n\nBlog\n\n\nArtigos\n\n\nTutoriais\n\n\n\n\nPesquisar na plataforma\n\n\n\n\n\n\n\nBlog da Asimov\nConfira as últimas notícias do mundo de tecnologia\n\n\n\n\n\n77 posts\n\n\n\nPesquisar\n\n\n\n\nCategoria\n\nTodas as categorias\nAnálise de Dados\nAplicações IA\nAutomações\nCiência de Dados\nDashboards\nProgramação Python\nTrading Quantitativo\nVisão Computacional\nVisualização de Dados\nWeb\nWeb Apps\n\n\n\n\n\n          Mais filtros\n        \n\n\nCategoria\n\nTodas as categorias\nAnálise de Dados\nAplicações IA\nAutomações\nCiência de Dados\nDashboards\nProgramação Python\nTrading Quantitativo\nVisão Computacional\nVisualização de Dados\nWeb\nWe

In [9]:
from langchain.pydantic_v1 import BaseModel, Field 
from typing import List 
from langchain_core.utils.function_calling import convert_to_openai_function

class BlogPost(BaseModel):
    """Informações sobre um post de blog"""
    titulo:str = Field(description="O título do post de blog")
    autor:str = Field(description="O autor do post de blog")
    data:str = Field(description="A data de publicação do post de blog")
    
class BlogSite(BaseModel):
    """Lista de blog posts de um site"""
    posts:List[BlogPost] = Field(description="Lista de posts de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Lista de blog posts de um site',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Lista de posts de blog do site',
    'type': 'array',
    'items': {'description': 'Informações sobre um post de blog',
     'type': 'object',
     'properties': {'titulo': {'description': 'O título do post de blog',
       'type': 'string'},
      'autor': {'description': 'O autor do post de blog', 'type': 'string'},
      'data': {'description': 'A data de publicação do post de blog',
       'type': 'string'}},
     'required': ['titulo', 'autor', 'data']}}},
  'required': ['posts']}}

In [11]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

prompt = ChatPromptTemplate.from_messages([
	("system", "Extraia da página todos os posts de blog com autor e data de publicação"),
	("user", "{input}")
])

chat = ChatOpenAI()

chain = prompt | chat.bind(functions=[tool_blog], function_call={'name':"BlogSite"}) | JsonKeyOutputFunctionsParser(key_name="posts")

In [12]:
chain.invoke({"input":page})

[{'titulo': 'A economia da inteligência artificial: como funciona o mercado da IA?',
  'autor': 'Renata Lopes',
  'data': '5 dias atrás'},
 {'titulo': 'História da IA: de Alan Turing aos dias atuais',
  'autor': 'Renata Lopes',
  'data': '6 dias atrás'},
 {'titulo': 'Processamento de Linguagem Natural: definição e exemplos de PLN',
  'autor': 'Renata Lopes',
  'data': '13 dias atrás'},
 {'titulo': 'O que é visão computacional e para que serve: guia para iniciantes',
  'autor': 'Renata Lopes',
  'data': '17 dias atrás'},
 {'titulo': 'O que é IA generativa? Definição, exemplos e aplicações práticas',
  'autor': 'Renata Lopes',
  'data': '24 dias atrás'},
 {'titulo': 'LLM: o que é e como funciona um grande modelo de linguagem',
  'autor': 'Renata Lopes',
  'data': '25 dias atrás'},
 {'titulo': 'Engenheiro de dados, cientista de dados e analista de dados: qual é a diferença?',
  'autor': 'Renata Lopes',
  'data': '1 mês atrás'},
 {'titulo': 'Análise de dados: o que é e como ingressar na ár